In [1]:
import openai
from qdrant_client import QdrantClient

### Embedding function

In [2]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )
    return response.data[0].embedding

### Retrieval function

In [3]:
qdrant_client = QdrantClient(url="http://localhost:6333")


In [4]:
def retrieve_data(query, qdrant_client, k=5):
    query_embedding = get_embedding(query)
    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "similarity_scores": similarity_scores,
    }

In [5]:
# retrieve_data("What kind of phone headphones can I get?", k=10).points[0].__dict__.keys()

# retrieve_data("What kind of phone headphones can I get?", k=10).points[0].payload.keys()

In [6]:
retrieved_context = retrieve_data("What kind of earphones can I get?", qdrant_client, k=10)

In [8]:
retrieved_context

{'retrieved_context_ids': ['B0CBMPG524',
  'B0B9FTVL58',
  'B0C142QS8X',
  'B0C6K1GQCF',
  'B0BNHVLF7G',
  'B0B14HTZ59',
  'B09WCFC5D9',
  'B0B67ZFRPC',
  'B0B7495RL6',
  'B0BS15TRJ3'],
 'retrieved_context': ['Open Ear Headphones, Bluetooth 5.3 Earbuds with 60H Playtime IPX7 Waterproof Wireless Earbuds Immersive Premium Sound True Wireless Open Ear Earbuds with Earhooks for Running, Walking and Workouts 【Open-ear Design Headphones】Feature with a new generation of true open-ear wireless earbuds design, the headphones can rest gently and firmly fit your ears without entering your ear canal, which will reduce stress and hearing loss after extended wear. There is no pinching of the auricle, no blockage of the ear canal, and no pain or damage to hearing. 【Powerful Stereo Sound】Equipped with 16.2 millimeters vibrating diaphragm speaker driver, bluetooth headphones providing pure balanced audio and clarity output for all music genres with soft audio and immerse yourself in the wonderful world

### Format retrieved context function

In [11]:
def process_context(context):

    formatted_context = ""

    for id, chunk in zip(context["retrieved_context_ids"], context["retrieved_context"]):
        formatted_context += f"- {id}: {chunk}\n"
    
    return formatted_context



In [12]:
preprocessed_context = process_context(retrieved_context)
print(preprocessed_context)

- B0CBMPG524: Open Ear Headphones, Bluetooth 5.3 Earbuds with 60H Playtime IPX7 Waterproof Wireless Earbuds Immersive Premium Sound True Wireless Open Ear Earbuds with Earhooks for Running, Walking and Workouts 【Open-ear Design Headphones】Feature with a new generation of true open-ear wireless earbuds design, the headphones can rest gently and firmly fit your ears without entering your ear canal, which will reduce stress and hearing loss after extended wear. There is no pinching of the auricle, no blockage of the ear canal, and no pain or damage to hearing. 【Powerful Stereo Sound】Equipped with 16.2 millimeters vibrating diaphragm speaker driver, bluetooth headphones providing pure balanced audio and clarity output for all music genres with soft audio and immerse yourself in the wonderful world of music. 【Hear Your Surroundings】Open earbuds that rest on your ears without covering them, you can hear your music and your surroundings at the same time. Whether you are cycling, walking, runn

### Create Prompt funtion

In [13]:
def build_prompt(preprocessed_context, question):

    prompt = f"""You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt

In [14]:
# print(build_prompt(preprocessed_context, "What kind of earphones can I get?"))

In [15]:
prompt = build_prompt(preprocessed_context, "What kind of earphones can I get?")
print(type(prompt))
print(prompt)


<class 'str'>
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
- B0CBMPG524: Open Ear Headphones, Bluetooth 5.3 Earbuds with 60H Playtime IPX7 Waterproof Wireless Earbuds Immersive Premium Sound True Wireless Open Ear Earbuds with Earhooks for Running, Walking and Workouts 【Open-ear Design Headphones】Feature with a new generation of true open-ear wireless earbuds design, the headphones can rest gently and firmly fit your ears without entering your ear canal, which will reduce stress and hearing loss after extended wear. There is no pinching of the auricle, no blockage of the ear canal, and no pain or damage to hearing. 【Powerful Stereo Sound】Equipped with 16.2 millimeters vibrating diaphragm speaker driver, bluetooth headphones providing p

### Generate Answer function

In [16]:
def generate_answer(prompt):

    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role":"system", "content": prompt},],
        temperature=0.5,
    )

    return response.choices[0].message.content

In [17]:
prompt

"You are a shopping assistant that can answer questions about the products in stock.\n\nYou will be given a question and a list of context.\n\nInstructions:\n- You need to answer the question based on the provided context only.\n- Never use word context and refer to it as the available products.\n\nContext:\n- B0CBMPG524: Open Ear Headphones, Bluetooth 5.3 Earbuds with 60H Playtime IPX7 Waterproof Wireless Earbuds Immersive Premium Sound True Wireless Open Ear Earbuds with Earhooks for Running, Walking and Workouts 【Open-ear Design Headphones】Feature with a new generation of true open-ear wireless earbuds design, the headphones can rest gently and firmly fit your ears without entering your ear canal, which will reduce stress and hearing loss after extended wear. There is no pinching of the auricle, no blockage of the ear canal, and no pain or damage to hearing. 【Powerful Stereo Sound】Equipped with 16.2 millimeters vibrating diaphragm speaker driver, bluetooth headphones providing pure 

In [18]:
print(generate_answer(prompt))

You can get various types of earphones from the available products, including:

1. Open Ear Headphones: Bluetooth 5.3 open-ear wireless earbuds that rest gently on your ears without entering the ear canal, providing safety and comfort for activities like running, walking, and workouts (Product B0CBMPG524).

2. Wireless In-Ear Earbuds: Bluetooth 5.3 earbuds with microphone, deep bass, LED power display, IPX7 waterproof rating, smart touch controls, and long playback time suitable for sports and everyday use (Product B0B9FTVL58).

3. Kids Headphones: Over-ear wired headphones for kids with volume limited to 94dB for hearing protection, adjustable and foldable design, compatible with devices with a 3.5mm jack (Product B0C142QS8X).

4. Wireless Bluetooth Earbuds for iPhone and Android: Bluetooth 5.1 earbuds with noise-canceling mic, touch control, deep bass, IPX7 waterproof, and long battery life with charging case (Product B0C6K1GQCF).

5. Bone Conduction Headphones: Wireless Bluetooth 5.

### Combined RAG Pipeline

In [19]:
def rag_pipeline(question, top_k=5):

    qdrant_client = QdrantClient(url="http://localhost:6333")

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    return answer

In [20]:
print(rag_pipeline("What different computer accessories ae there?"))

The available computer accessories include:

1. Wireless Keyboard and Mouse Combo (B09PYFMTBF) - A 2.4G wireless set with retro round keys, slim and quiet design, compatible with Windows computers and laptops.

2. Bextsrack Monitor Stand Riser (B09XCKYXR8) - A metal desktop stand for monitors, laptops, printers, and other devices that helps elevate the screen for better ergonomics and prevents overheating.

3. Cleaner Kit for AirPods Pro and other devices (B0B7495RL6) - A 4-in-1 cleaning pen designed for cleaning earbuds, keyboards, mice, cellphones, laptops, and cameras.

These accessories help enhance the computer setup by providing input devices, ergonomic support, and cleaning tools.
